# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
# Импортируем нужные нам библиотеки

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score 

import numpy as np

Теперь выгрузим файл с данными и посмотрим его

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
df.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Исcледование мультиколлинеарности факторов

Для исследования *мультиколлинеарности* стоит построить корреляционную матрицу по входным данным, так как созависимые параметры будут негативно сказываться на обучении модели, дублируя друг друга. При этом не будут нести информацию сами по себе, что может привести к переобучению модели.

In [4]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,calls,minutes,messages,mb_used,is_ultra
calls,1,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1,0.17311,0.280967,0.206955
messages,0.177385,0.17311,1,0.195721,0.20383
mb_used,0.286442,0.280967,0.195721,1,0.198568
is_ultra,0.207122,0.206955,0.20383,0.198568,1


> Как мы видим, все данные независимы друг от друга, **кроме минут и звонков**. Они имеют практиески прямую зависимость - *степень корреляции* у них составляет - **0.982**. 

Таким образом, мы можем вычеркнуть столбец со звонками. Так как физически важнее не количество звонков как таковых - они могут иметь и нулевую длительность. А количество фактически использованных минут, которые являются определеющим фактором для выбора модели.

In [5]:
df = df.drop(['calls'],axis =1)

### Выводы

- Данные выгружены без ошибок;
- Данные полные, пропусков нет;
- Проведено исследование на мультиколлинеарность;
- Принятно решение не включать в данные столбец с количеством звонков пользователей.

## Разбейте данные на выборки

Разобьем выборки на обучающую, валидационную и тестовую выборки.

In [6]:
# Для начала разобьем данные на две выборки. Первое разбиение - на обучающую и валидационную в соотношении 60/40
data_train,data_valid = train_test_split(
    df,test_size = 0.60, random_state=12345)

# Затем полученную валидационную разобьем пополам для получения тестовой и валидационной
data_test,data_valid = train_test_split(data_valid,test_size = 0.50, random_state=42)

Теперь посмотрим на размер получившихся выборок.

In [7]:
print(f"{np.shape(data_train)}\n{np.shape(data_valid)}\n{np.shape(data_test)}")


(1285, 4)
(965, 4)
(964, 4)


Всё отлично, выборки разделились довольно равномерно. 

Суммарно как раз получилось 3214 записей - как раз столько же, как и в изначальном наборе данных, значит, дублирований нет.

Так же определим обозначим целевые,валидационные признаки и признаки для обучения модели.

In [8]:
# Признаки для обучающей выборки
features_train = data_train.drop(['is_ultra'],axis =1)
target_train = data_train['is_ultra']

# Признаки для валидационной выборки
features_valid = data_valid.drop(['is_ultra'],axis =1)
target_valid = data_valid['is_ultra']

# Признаки для тестовой выборки
features_test = data_test.drop(['is_ultra'],axis =1)
target_test = data_test['is_ultra']

## Исследование моделей

### Дерево решений

Исследуем первый вариант модели - *дерево решений*

Подберем лучшую модель для каждого случая, для этого в цикле посмотрим **лучшую глубину** для дерева решений.

In [9]:
best_model = None
best_result = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train,target_train) 
  #  predictions = model.predict(features_valid) 
    predictions_valid = model.predict(features_valid)   # СКОРРЕКТИРОВАЛ ТУТ ОПИСКУ
    result = accuracy_score(target_valid, predictions_valid) 
    if result > best_result:
        best_model = model
        best_result = result
        
print("Accuracy лучшей модели:", best_result)
print("best model:",model)

Accuracy лучшей модели: 0.8
best model: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')


### Логистическя регрессия

In [10]:
model = LogisticRegression(random_state=12345)
model.fit(features_train,target_train)
result = model.score(features_valid,target_valid)

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7419689119170985


### Лес случайных деревьев

Для леса случайных деревьев посмотрим два гиперпараметра - лучшую максимальную глубину и количество деревьев в лесу.

In [11]:
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
best_result_score = 0
best_criterion = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        for criteria in ['gini','entropy']:
            model = RandomForestClassifier(random_state = 12345, n_estimators=est, max_depth=depth, criterion=criteria)
            model.fit(features_train,target_train)
            predictions_valid = model.predict(features_valid)
            result = mean_squared_error(target_valid,predictions_valid)**0.5
            result_score = accuracy_score(target_valid, predictions_valid) 
            if result < best_result:
                best_model = model
                best_result = result
                best_est = est
                best_depth = depth
                best_result_score = result_score
                best_criterion = criteria

print("RMSE наилучшей модели на валидационной выборке:", best_result,
      "Количество деревьев:", best_est,
      "Максимальная глубина:", depth,
      "Лучший критерий",best_criterion)
print("Accuracy лучшей модели:",best_result_score)

RMSE наилучшей модели на валидационной выборке: 0.42218283002617707 Количество деревьев: 20 Максимальная глубина: 10 Лучший критерий gini
Accuracy лучшей модели: 0.8217616580310881


### Вывод

- *Логистическая регрессия* хоть и работает быстро, но точность предсказательной модели на валидационной выборке ниже всех прочих моделей - **0.74**;
- *Дерево решений* дает на валидационной выборке средний результат **0.80** при глубине решающего дерева **9** и выборе критерия **Джини**;
- Лучший результат показала модель *леса случайных деревьев* со *следующими гиперпараметрами*:
 - Количество деревьев: 20;
 - Максимальная глубина: 10;
- Точность данной предсказательной модели на валидационной выборке составила **0.82**

## Проверьте модель на тестовой выборке

In [14]:
model = RandomForestClassifier(random_state = 12345, n_estimators=10, max_depth=10, criterion = 'entropy')

model.fit(features_train,target_train)

predictions_test = model.predict(features_test)
result_score = accuracy_score(target_test, predictions_test) 

print("Accuracy")
print("Тестовая выборка:", result_score)

Accuracy
Тестовая выборка: 0.7904564315352697


### Вывод

- Точность предсказания получилась **0.79**;
- Модель с лесом случайных деревеьев отработала успешно;
- В целом при варьировании гиперпараметров удалось найти подходящую модель,выдающую достаточно неплохое значение точности на тестовой выработке.

## (бонус) Проверьте модели на адекватность

In [15]:
#Импортируем фиктивный классификатор
from sklearn.dummy import DummyClassifier
# Создадим модель, где будет в качестве предиктора использоваться самое частное значение
d_c = DummyClassifier(strategy = 'most_frequent', random_state = 12345)
# Обучим ее на тренеровочной выборке
d_c.fit(features_train,target_train)
# и посмотрим долю правильных ответов
result_valid = d_c.score(features_valid,target_valid)
result_test = d_c.score(features_test,target_test)


print('Accuracy фиктивного классификатора на валидационной выборке:', result_valid)
print('Accuracy фиктивного классификатора на тестовой выборке:', result_test)


Accuracy фиктивного классификатора на валидационной выборке: 0.7316062176165803
Accuracy фиктивного классификатора на тестовой выборке: 0.6846473029045643


### Выводы

- На валидационной выборке фиктивный классификатор показал значение **0.73** - примерно такое же, как и на логистической регрессии;
- На тестовой выборке мы получили значение точности **0.68**, что ниже **0.79**, полученное при использовании модели случайного леса;
- Таким образом, наша модель является рабочей и адекватной.
    